In [1]:
!pip -q install -U datasets transformers accelerate evaluate scikit-learn numpy
!pip -q install -U evaluate
import os, zipfile, numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer
)
import evaluate



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
ds = load_dataset("ailsntua/QEvasion")  # train: 3448, test: 308


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

In [15]:
ds = load_dataset("ailsntua/QEvasion")
tok = AutoTokenizer.from_pretrained("Qwen/Qwen3-Embedding-0.6B")

Q_COL = "question"
A_COL = "interview_answer"

def add_len(batch):
    enc = tok(
        batch[Q_COL], batch[A_COL],
        truncation=False,      # để đo length thật, không cắt
        padding=False,
        return_length=True,    # tokenizer trả về length [web:200]
    )
    return {"n_tokens": enc["length"]}

# tính length cho cả train + test
train_l = ds["train"].map(add_len, batched=True, batch_size=256)
test_l  = ds["test"].map(add_len, batched=True, batch_size=256)

def summary(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()),
        "p50": int(np.percentile(arr, 50)),
        "p90": int(np.percentile(arr, 90)),
        "p95": int(np.percentile(arr, 95)),
        "p99": int(np.percentile(arr, 99)),
        "max": int(arr.max()),
        "mean": float(arr.mean()),
    }

print("Train token length stats:", summary(train_l["n_tokens"]))
print("Test  token length stats:", summary(test_l["n_tokens"]))

# gợi ý max_length theo percentile (thường chọn p95 hoặc p99 tùy VRAM)
p95 = int(np.percentile(train_l["n_tokens"], 95))
p99 = int(np.percentile(train_l["n_tokens"], 99))
print("Suggested max_length (p95):", p95)
print("Suggested max_length (p99):", p99)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Map:   0%|          | 0/3448 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

Train token length stats: {'count': 3448, 'min': 5, 'p50': 273, 'p90': 853, 'p95': 1104, 'p99': 1634, 'max': 2465, 'mean': 376.29147331786544}
Test  token length stats: {'count': 308, 'min': 13, 'p50': 379, 'p90': 835, 'p95': 1006, 'p99': 1349, 'max': 1599, 'mean': 413.3279220779221}
Suggested max_length (p95): 1104
Suggested max_length (p99): 1634


In [20]:
train_ds = ds["train"]
val_ds   = ds["test"]   # có label => dùng làm validation

label_list = ["Ambivalent", "Clear Non-Reply", "Clear Reply"]
label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}

def encode_label(ex):
    ex["labels"] = label2id[ex["clarity_label"]]
    return ex

train_ds = train_ds.map(encode_label)
val_ds   = val_ds.map(encode_label)

Map:   0%|          | 0/3448 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [25]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_name = "FacebookAI/xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Q_COL = "question"
A_COL = "interview_answer"

def preprocess(batch):
    # Tokenize sequence pair: (text, text_pair)
    return tokenizer(
        batch[Q_COL],
        batch[A_COL],
        truncation=True,
        max_length=512,
    )

train_tok = train_ds.map(preprocess, batched=True)
val_tok   = val_ds.map(preprocess, batched=True)

# Giữ cột cần cho Trainer: input_ids/attention_mask/labels
keep = {"input_ids", "attention_mask", "labels"}
train_tok = train_tok.remove_columns([c for c in train_tok.column_names if c not in keep])
val_tok   = val_tok.remove_columns([c for c in val_tok.column_names if c not in keep])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Map:   0%|          | 0/3448 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# metric (macro F1)
f1 = evaluate.load("f1")  # load once (đúng best practice) [web:84]

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"macro_f1": f1.compute(predictions=preds, references=labels, average="macro")["f1"]}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
)

args = TrainingArguments(
    output_dir="deberta_large_clarity",
    learning_rate=2e-5,             # large model thường LR thấp hơn
    per_device_train_batch_size=32,  # H200 có thể lên 64 tùy max_length
    per_device_eval_batch_size=32,
    num_train_epochs=10,            # để early stopping tự cắt

    lr_scheduler_type="cosine",
    warmup_ratio=0.06,
    weight_decay=0.01,
    max_grad_norm=1.0,

    eval_strategy="steps",
    eval_steps=100,
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    save_total_limit=2,

    bf16=True,                      # H200 hỗ trợ BF16 ngon, ổn hơn fp16
    gradient_checkpointing=True,     # nếu muốn tăng max_length/batch
    dataloader_num_workers=4,
    report_to="none",
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_164/1062934490.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Macro F1
100,No log,0.822912,0.291000
200,No log,0.786431,0.267185
300,No log,0.735548,0.528974
400,No log,0.687350,0.568019
500,0.808200,0.753493,0.555972
600,0.808200,0.739630,0.583500
700,0.808200,0.740419,0.611496
800,0.808200,0.897437,0.578015
900,0.808200,0.932347,0.617824


In [ ]:
# tokenize test (không có labels)
test_tok = test_ds.map(preprocess, batched=True, remove_columns=test_ds.column_names)

pred = trainer.predict(test_tok)
pred_ids = np.argmax(pred.predictions, axis=-1)
pred_labels = [id2label[i] for i in pred_ids]

# ghi file extensionless: "prediction"
with open("prediction", "w", encoding="utf-8") as f:
    for lab in pred_labels:
        f.write(lab + "\n")

print("Wrote:", os.path.abspath("prediction"))
print("Num lines:", sum(1 for _ in open("prediction", "r", encoding="utf-8")))


In [ ]:
zip_path = "submission.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    z.write("prediction", arcname="prediction")  # quan trọng: arcname không có folder

print("Zipped:", os.path.abspath(zip_path))

# sanity check: xem zip có đúng 1 file prediction không
with zipfile.ZipFile(zip_path, "r") as z:
    print("ZIP content:", z.namelist())
